In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [3]:
wiki = 'https://en.wikipedia.org'

In [4]:
link_list = []
for row in My_table.tbody.findAll('tr'):
    links = row.find_all('a')[0]
    #print(links['href'])
    link = wiki+links['href']
    link_list.append(link)
    #print(link)

In [5]:
table_rows = My_table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


In [6]:
data = pd.DataFrame(res)
data.columns = ['2017 rank', 'City', 'State', '2017 estimate', '2010 Census',
                                                         'Change', '2016 land area Sq Mi','2016 land area Sq Km',
                 '2016 population density/sq Mi', '2016 population/sq Km', 'Location']

In [7]:
for i, row in data.iterrows():
    row['City'] = re.sub(r'\[.*?\]', '', row['City'])
    row['2016 land area Sq Mi'] = re.sub("\D", "", row['2016 land area Sq Mi'])
    row['2016 land area Sq Km'] = re.sub("\D", "", row['2016 land area Sq Km'])
    row['2016 population density/sq Mi'] = re.sub("\D", "", row['2016 population density/sq Mi'])
    row['2016 population/sq Km'] = re.sub("\D", "", row['2016 population/sq Km'])

In [8]:
data['Wikipedia Link'] = link_list[1:]

In [24]:
web = []
for l in link_list[1:]:
    website_url = requests.get(l).text
    soup = BeautifulSoup(website_url,'lxml')
    tables = soup.find_all('table')
    k = tables[0].findAll('a')
    if(len(k)<20):
        k = tables[1].findAll('a')
    if(len(k)<20):
        k = tables[2].findAll('a')
    w = k[len(k) - 1]['href']
    if "wikidata" in w:
        w = k[len(k) - 2]['href']
    web.append(w)

In [26]:
data['City Website'] = web

In [ ]:
data.to_csv('my_data.csv', index = False)